In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile

In [2]:
from qiskit.visualization import plot_histogram

In [3]:
from qiskit.providers import basic_provider

Source for the code: 
Oliveira, David & Ramos, Rubens. (2007). Quantum bit string comparator: Circuits and applications. Quantum Computers and Computing. 7. 

In [4]:
def bit_compare():
    qr = QuantumRegister(2, "bits")
    aux = QuantumRegister(2, "aux")
    
    qc = QuantumCircuit(qr, aux)
    qc.x(qr[1])
    qc.mcx(qr, aux[0])
    qc.x(qr[0])
    qc.x(qr[1])
    qc.mcx(qr, aux[1])
    qc.x(qr[0])
    
    return qc

In [5]:
print(bit_compare())

                  ┌───┐     ┌───┐
bits_0: ───────■──┤ X ├──■──┤ X ├
        ┌───┐  │  ├───┤  │  └───┘
bits_1: ┤ X ├──■──┤ X ├──■───────
        └───┘┌─┴─┐└───┘  │       
 aux_0: ─────┤ X ├───────┼───────
             └───┘     ┌─┴─┐     
 aux_1: ───────────────┤ X ├─────
                       └───┘     


In [6]:
## Defining the encode function to encode the given string by applying an x gate if bit is one and doing nothing if bit is zero
def encode(bit):
    qr = QuantumRegister(1, "number")
    qc = QuantumCircuit(qr)
    if (bit == "1"):
        qc.x(qr[0])
    return qc

## We need to use this function again and again for each bit

In [7]:
def compare_bitstring(bitstring_a, bitstring_b, exec=True):
    bits = len(bitstring_a)
    qra = QuantumRegister(bits, "a")
    qrb = QuantumRegister(bits, "b")
    qraux = QuantumRegister(2*bits, "aux")
    qrint = QuantumRegister(bits-1, "int")
    cr = ClassicalRegister(2)

    qc = QuantumCircuit(qra, qrb, qraux, qrint, cr)

    for i in range(bits):
        qc.append(encode(bitstring_a[i]), [qra[i]])
        qc.append(encode(bitstring_b[i]), [qrb[i]])
        qc.append(bit_compare(), [qra[i], qrb[i], qraux[2*i], qraux[2*i+1]])
        
        if i < bits-1:
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
            qc.mcx([qraux[2*i], qraux[2*i+1]], qrint[i])
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
        
    for i in range(0, bits-1):
        qc.mcx([qraux[2*(-i-1)],  qrint[-i]], qraux[2*(-i)])
        qc.mcx([qraux[2*(-i-1)+1],  qrint[-i]], qraux[2*(-i)+1])
        
    qc.measure(qraux[0], cr[0])
    qc.measure(qraux[1], cr[1])
    
    if exec:
        # Tell Qiskit how to simulate our circuit
        backend = basic_provider.BasicProvider().get_backend('basic_simulator')

        # Do the simulation, returning the result
        res = transpile(qc,backend)
        job=backend.run(res)
        results=job.result()

        # get the probability distribution
        counts = results.get_counts()

        return counts
    else:
        return qc

In [8]:
print(compare_bitstring('101','110'))

{'01': 1024}


In [9]:
def to_bits(x):
    y=bin(x)[2:]
    return y

In [13]:
def less_than_k(lst,k):
    l=[]
    for i in lst:
        #k=compare_bitstring("010", "100")
        if((list(compare_bitstring(to_bits(i), to_bits(k)).keys())[0]) == '10'):
            l.append(i)
    return l
        
less_than_k([1,2,3,4,5],6)

[2]

Problem includes not correct output for the cases (110,1